In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
#경로 설정
directory= 'C:\\Users\\USER\\PycharmProjects\\ERC\\'
os.chdir(directory)

annotation 취합
---

In [ ]:
ann = os.path.join(directory, 'data/20/annotation')
files = os.listdir(ann)
filescnt_wav = [i for i in files if i.endswith('.csv')]

df_all = pd.DataFrame()
for i in range(0, len(filescnt_wav)) :
    file = filescnt_wav[i]
    df = pd.read_csv(os.path.join(ann, file), encoding = 'utf-8')
    df['filename'] = file.split('_eval')[0]
    df_all = pd.concat([df_all, df])
sess_df = df_all.drop([0],axis = 0)

wav/txt 취합
---

In [ ]:
def search(dirname):
    result = []
    filenames = os.listdir(dirname)
    for filename in filenames:
        full_filename = dirname + '//' + filename
        result.append(full_filename)    
    return result

In [ ]:
wav_path = os.path.join(directory, 'data/20/wav')
wav_sess = search(wav_path)

In [ ]:
txt_files = []
wav_files = []

for i in wav_sess:
    filenames = os.listdir(i)
    direc = i[42:]
    for filename in filenames:
        full_filename = direc + '//' + filename
        if full_filename[-3:] == 'wav':
            wav_files.append(full_filename)
        elif full_filename[-3:] == 'txt':
            txt_files.append(full_filename)
        else:
            print('wrong!')

In [ ]:
total = pd.DataFrame({'Audio': wav_files,
                      'Text': txt_files,
                      'Emotion': wav_files,
                      'Arousal': wav_files,
                      'Valence': wav_files})

전체 데이터 취합
---

In [ ]:
path = directory+'data/20/'
for i in tqdm(range(total.shape[0])):
    directory = os.path.join(path, total.iloc[i, 1])
    with open (directory, 'r') as text:
        string = text.readlines()
    total.iloc[i, 1] = string[0].replace('\n', '')

In [ ]:
for i in tqdm(range(total.shape[0])):
    file_name = total.iloc[i, 2][16:-4]
    total.iloc[i, 2] = sess_df[sess_df['Segment ID'] == file_name].iloc[0, 4]
    total.iloc[i, 3] = sess_df[sess_df['Segment ID'] == file_name].iloc[0, 6]
    total.iloc[i, 4] = sess_df[sess_df['Segment ID'] == file_name].iloc[0, 5]

In [ ]:
total

In [ ]:
for i in range(total.shape[0]):
    text = total.iloc[i, 1]
    text = text.replace('c/','').replace('n/','').replace('N/','').replace('u/','').replace('l/','')
    text = text.replace('b/','').replace('*','').replace('+','').replace(',','')
    total.iloc[i, 1] = text

In [ ]:
list_id = []
list_sex = []
for i in range(total.shape[0]):
    list_id.append(total.iloc[i, 0][16:-4])
    list_sex.append(total.iloc[i, 0][39])
    
total['id'] = list_id
total['Sex'] = list_sex

In [ ]:
total = total[['id', 'Sex', 'Emotion', 'Arousal', 'Valence']]
total.to_csv('C:\\Users\\USER\\PycharmProjects\\ERC\\data\\total.csv', index=False, encoding="utf-8-sig") 
total.head()

불러오기
---

In [10]:
sig_n = 'temp' #eda, ibi, temp

signal = pd.read_csv('data/t_{}.csv'.format(sig_n))
total = pd.read_csv('data/total.csv')

In [4]:
from sklearn.utils import resample

def balancing(list_emotion, total_new):
    df_balance = pd.DataFrame()
    for emo in list_emotion:
        data_emo = total_new[total_new['Emotion'] == emo]
        data_ds = resample(data_emo, n_samples=200, random_state=42)
        df_balance = pd.concat([df_balance, data_ds])
    return df_balance

In [5]:
total['Emotion'].loc[(total['Emotion'] == 'happy;neutral')] = 'happy'
total['Emotion'].loc[(total['Emotion'] == 'angry;neutral')] = 'angry'
total['Emotion'].loc[(total['Emotion'] == 'neutral;sad')] = 'sad'
total['Emotion'].loc[(total['Emotion'] == 'surprise;neutral')] = 'surprise'
total['Emotion'].loc[(total['Emotion'] == 'neutral;disqust')] = 'disqust'
total['Emotion'].loc[(total['Emotion'] == 'neutral;fear')] = 'fear'

list_emotion = ['neutral', 'happy', 'surprise', 'angry', 'sad'] #, 'disqust', 'fear']
condition = total['Emotion'].isin(list_emotion)
total_new = total[condition]

C:\Users\USER\AppData\Local\Temp\ipykernel_7300\3348865692.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total['Emotion'].loc[(total['Emotion'] == 'happy;neutral')] = 'happy'
C:\Users\USER\AppData\Local\Temp\ipykernel_7300\3348865692.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total['Emotion'].loc[(total['Emotion'] == 'angry;neutral')] = 'angry'
C:\Users\USER\AppData\Local\Temp\ipykernel_7300\3348865692.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [11]:
all_data = total_new.merge(signal, how='inner', on='id')
all_data = all_data.fillna(0)

df_balance = balancing(list_emotion, all_data)
df_balance.to_csv('data/{}.csv'.format(sig_n), index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.scatterplot(data=df_balance, x='Arousal', y='Valence', hue='Emotion', alpha=.3, s=80)
plt.savefig('img/arou_vlae.png')
plt.show()

데이터 조합
---

In [28]:
sig_n = 'eda' #eda, ibi, temp
sig_n2 = 'ibi'

signal = pd.read_csv('data/{}.csv'.format(sig_n))
signal2 = pd.read_csv('data/{}.csv'.format(sig_n2))

signal_new = signal.merge(signal2, how='inner', on=['id', 'Sex', 'Emotion', 'Arousal', 'Valence'] )

signal_balance = balancing(list_emotion, signal_new)
signal_balance.to_csv('data/{}_{}.csv'.format(sig_n, sig_n2), index=False)

In [29]:
signal_new

,id,Sex,Emotion,Arousal,Valence,eda1,eda2,eda3,eda4,eda5,...,ibi59,ibi60,ibi61,ibi62,ibi63,ibi64,ibi65,ibi66,ibi67,ibi68
0,Sess14_script02_User027F_025,F,neutral,2.9,2.3,4.735661,4.729254,4.713878,4.693377,4.688251,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Sess10_script06_User019F_001,F,neutral,2.6,2.2,1.861127,1.852158,1.859846,1.859846,1.858564,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Sess14_script06_User028M_001,M,neutral,3.0,3.0,5.445272,5.414522,5.386335,5.370960,5.372241,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Sess35_script03_User070F_007,F,happy,3.4,3.6,0.084446,0.079328,0.087005,0.088284,0.087005,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Sess25_script01_User049M_043,M,happy,3.2,3.7,2.658107,2.665786,2.676025,2.697783,2.719540,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,Sess30_script02_User059F_026,F,sad,2.9,2.4,0.080996,0.086121,0.088684,0.087403,0.080996,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
632,Sess19_script05_User038F_014,F,sad,3.3,2.9,5.927680,5.832138,5.753976,5.701442,5.670690,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
633,Sess14_script04_User027F_022,F,sad,3.2,3.1,7.137491,7.239985,7.313012,7.345041,7.339917,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
634,Sess27_script02_User053M_026,M,sad,2.9,2.5,5.196572,5.177369,5.169687,5.177369,5.208095,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
sig_n3 = 'temp'

signal3 = pd.read_csv('data/{}.csv'.format(sig_n3))

signal_new = signal_new.merge(signal3, how='inner', on=['id', 'Sex', 'Emotion', 'Arousal', 'Valence'] )
signal_balance = balancing(list_emotion, signal_new)
signal_balance.to_csv('data/{}_{}_{}.csv'.format(sig_n, sig_n2, sig_n3), index=False)